In [10]:
import requests
import json
from chromadb import Client
from sentence_transformers import SentenceTransformer

# Initialize Here Maps API key
here_api_key = "I9TdZQPeR9GC_tibSgdY2S3hR_JAWj1g37_Kx3KR-i4"

# Initialize ChromaDB client and embedding model
client = Client()
model = SentenceTransformer('all-MiniLM-L6-v2')  # Example model; adjust as needed

# Create or access ChromaDB collection
collection = client.get_or_create_collection(name="address_data_here")

# Function to get address data from Here Maps
def fetch_address_data(query):
    url = f"https://geocode.search.hereapi.com/v1/geocode"
    params = {
        "q": query,
        "apiKey": here_api_key,
    }
    response = requests.get(url, params=params)
    data = response.json()
    print(data)
    
    if data['items']:
        # Extract the first result
        result = data['items'][0]
        address = result['address']
        return {
            "street": address.get("label", ""),
            "city": address.get("city", ""),
            "state": address.get("state", ""),
            "postal_code": address.get("postalCode", ""),
            "country": address.get("countryName", ""),
            "lat": result["position"]["lat"],
            "lng": result["position"]["lng"],
        }
    else:
        return None

# Example query to get and store address data
query = "23-25 avenue du Dr. Lannelongue,Paris Cedex 14"
address_data = fetch_address_data(query)
print(address_data)

if address_data:
    # Construct embedding from full address
    address_text = f"{address_data['street']}, {address_data['city']}, {address_data['state']}, {address_data['postal_code']}, {address_data['country']}"
    embedding = model.encode(address_text).tolist()
    
    # Prepare metadata with Here Maps data
    metadata = {
        "street": address_data["street"],
        "city": address_data["city"],
        "state": address_data["state"],
        "postal_code": address_data["postal_code"],
        "country": address_data["country"],
        "latitude": address_data["lat"],
        "longitude": address_data["lng"]
    }
    
    # Store in ChromaDB
    unique_id = f"{address_data['street']}_{address_data['postal_code']}"
    collection.add(
        documents=[address_text],
        metadatas=[metadata],
        ids=[unique_id],
        embeddings=[embedding]
    )

# Query ChromaDB by example input
query_input = "23-25 avenue du Dr. Lannelongue, Paris Cedex 14"  # Example partial address
query_embedding = model.encode(query_input).tolist()
results = collection.query(query_embeddings=[query_embedding], n_results=1)

# Display results
for metadata in results['metadatas']:
    metadata_entry = metadata[0]
    print(f"Street: {metadata_entry.get('street', 'N/A')}")
    print(f"City: {metadata_entry.get('city', 'N/A')}")
    print(f"State: {metadata_entry.get('state', 'N/A')}")
    print(f"Postal Code: {metadata_entry.get('postal_code', 'N/A')}")
    print(f"Country: {metadata_entry.get('country', 'N/A')}")
    print(f"Latitude: {metadata_entry.get('latitude', 'N/A')}")
    print(f"Longitude: {metadata_entry.get('longitude', 'N/A')}")


Insert of existing embedding ID: Avenue du Docteur Lannelongue, 75014 Paris, France_75014
Add of existing embedding ID: Avenue du Docteur Lannelongue, 75014 Paris, France_75014


{'items': [{'title': 'Avenue du Docteur Lannelongue, 75014 Paris, France', 'id': 'here:af:streetsection:.0juVWW4pnjiZjujfDF.FD', 'resultType': 'street', 'address': {'label': 'Avenue du Docteur Lannelongue, 75014 Paris, France', 'countryCode': 'FRA', 'countryName': 'France', 'stateCode': 'IDF', 'state': 'Île-de-France', 'county': 'Paris', 'city': 'Paris', 'district': '14e Arrondissement', 'street': 'Avenue du Docteur Lannelongue', 'postalCode': '75014'}, 'position': {'lat': 48.8186, 'lng': 2.33101}, 'mapView': {'west': 2.3293, 'south': 48.8182, 'east': 2.33271, 'north': 48.81899}, 'scoring': {'queryScore': 0.69, 'fieldScore': {'city': 1.0, 'district': 0.67, 'streets': [0.95]}}}]}
{'street': 'Avenue du Docteur Lannelongue, 75014 Paris, France', 'city': 'Paris', 'state': 'Île-de-France', 'postal_code': '75014', 'country': 'France', 'lat': 48.8186, 'lng': 2.33101}
Street: Avenue du Docteur Lannelongue, 75014 Paris, France
City: Paris
State: Île-de-France
Postal Code: 75014
Country: France
